## Name     :  Akhil Amit Senroy
UBIT       :  akhilami <br>
Person# :  50288588


# Imports and APIs

In [24]:
import tweepy
import csv 
from nytimesarticle import articleAPI
from bs4 import BeautifulSoup
import requests

ConsumerKey = "7nOjD4MCGqdpwvfgdGNs65m22"
ConsumerSecret = "wpEO61KmNh3ddYfhULFsd2cdT8sFjYN6zlyQasCCqIzUIXl4OK"
AccessKey = "464870798-75FP4gx8AO6Bil2Gk31Ds1x79BUlVFJVc6FQq0OO"
AccessSecret = "vQXhHdINSjYEALuItjDuiDGuBkCWy7z6zSmzgUGlzDSKL"
auth = tweepy.OAuthHandler(ConsumerKey,ConsumerSecret)
auth.set_access_token(AccessKey,AccessSecret)
api = tweepy.API(auth)

nytapi = articleAPI('AJBvphX6dlpTN9T54u1XkAbg9cUjxA0r')

# Data Collection

### Twitter 

In [ ]:
csvFile = open('data/tw/sample/twpiano.csv', 'w')
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search,q=" piano -filter:retweets",\
                           lang="en",\
                           count = 2000,\
                           include_entities=True,\
                           since_id=2019-1-1).items():
    if not tweet.retweeted:
        print(tweet.created_at, tweet.text)
        csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])


In [18]:
import pandas as pd
colnames = ['date', 'tweets']
data = pd.read_csv('data/tw/sample/twsong.csv', names=colnames,index_col=False)
col = data['tweets']
txt = col.to_string()
txt_file = open('data/tw/sample/twsong.txt','w')
txt_file.write(txt)
txt_file.close()
# saved_column = df.column_name

### New York Times

In [8]:
import csv
import time 

linksSet = {""}
topics = ["music","song","guitar","violin","piano","sing","orchestra"]


for topic in topics:
    num=1
    for pageno in range(0,20):
#         print(topic)
        articles = nytapi.search(q = topic, begin_date = 20190101 , page = pageno )
        if 'response' in articles:
            n = len(articles['response']['docs'])
            for j in range(n):
                url = articles['response']['docs'][j]['web_url']
                linksSet.add(url)
#                 print(url)
                try:
                    data = requests.get(url)
                    soup = BeautifulSoup(data.content, 'html.parser')
                    prettySoup = soup.prettify()
                    f = open('data/nyt/sample/'+topic+str(num)+'.txt', 'w')
                    for k in range((len(soup.find_all('p')))-3):
                        f.write(soup.find_all('p')[k].get_text())
                    f.close()
                except:
                    continue
                num+=1
                

    time.sleep(10)
        

### CommonCrawl

In [2]:
with open("CommonCrawlEnv/links.txt") as input:
    links = input.read()
    num=0
    for link in links.split("\n\n"):
        try:
            data = requests.get(link)
            soup = BeautifulSoup(data.content, 'html.parser')
            prettySoup = soup.prettify()
            f = open('data/cc/sample/cc'+str(num)+'.txt', 'w')
            for k in range((len(soup.find_all('p')))-3):
                f.write(soup.find_all('p')[k].get_text())
            f.close()
        except:
            continue
        num+=1

In [5]:
import os
import glob
from os.path import join, getsize

read_files = glob.glob('data/cc/sample/cc*')
for f in read_files:
    if getsize(f)==0:
        os.remove(f)

# Data Cleaning

In [8]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
stopwords=set(stopwords.words('english'))
# https://stackoverflow.com/questions/17749058/combine-multiple-text-files-into-one-text-file-using-python
import glob
i=0
read_files = glob.glob("data/cc/sample/*")

for f in read_files:
    with open("data/cc/clean_sample/cleanCC"+str(i)+".txt", "w+") as newf:
        i+=1
        with open(f, "r") as infile:
            file=infile.read()
    
            words=word_tokenize(file)

            # include only words no numerical or special character
            words_rem_punct=[word for word in words if word.isalpha()]

            # removing all the stop words 
            meaningful_words=[x for x in words_rem_punct if not x in stopwords]
            meaningful_words[:10]

            #removing words with only one character since they will add any meaning to the data
            meaningful_words=[z for z in meaningful_words if not len(z)==1]
            # meaningful_words[:10]

            # stemming of the data(removing words that convey the same meaning)
            stemmer=PorterStemmer()
            stemmed_words=[stemmer.stem(words) for words in meaningful_words]
            # print(stemmed_words)

            #writting stemmed words into a file

            for w in stemmed_words:
                newf.write(w.lower()+" ")
    



# Combine Multiple Files

In [56]:
# https://stackoverflow.com/questions/17749058/combine-multiple-text-files-into-one-text-file-using-python
import glob

read_files = glob.glob("data/cc/outputWordCoOccur/p*")

with open("data/cc/outputWordCoOccur/combined_output.txt", "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())

# Convert Text files to CSV

In [57]:
import pandas as pd
data = pd.read_csv('data/cc/outputWordCoOccur/combined_output.txt', sep="\t", header=None)
data.to_csv('data/cc/outputWordCoOccur/combined_output.csv', sep=',',index=False,header=False)

# Sort in Descending order of Max Frequency

In [58]:
import csv

with open('data/cc/outputWordCoOccur/combined_output.csv', 
          mode='rt') as f, open('data/cc/WordCoOccur.csv',
                                'w') as final, open('data/cc/toptenWordCoOccur.csv','w') as topten:
    writer = csv.writer(final)
    writer2 = csv.writer(topten)
    reader = csv.reader(f)
    _ = next(reader)
    sortedlist = sorted(reader, key=lambda row: int(row[1]),reverse=True)
    for row in sortedlist:
        writer.writerow(row)
    for row in sortedlist[:10]:
        writer2.writerow(row)
        
        

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br> 
## Helpers

In [51]:
#http://www.science.smith.edu/dftwiki/index.php/Hadoop_Tutorial_2_--_Running_WordCount_in_Python
#import sys
#for line in sys.stdin:
topics = ["music","song","guitar","violin","piano","sing","orchestra"]

line = " first report peopl magazin the actress musician wed last juli upstat new york thi first marriag bob dylan one incorpor stage banter recent year he rare even speak introduc band but chang last night someon broke photo polici show vienna austria after sing one vers blowin wind dylan stop said audienc take pictur take pictur we either play pose okay befor there point mani young rapper live becom infatu mf doom joey badass mine volum special herb compil beat debut album milo scour fileshar servic kazaa find doom loosi year odd futur compatriot tyler the creator earl sweatshirt look like toddler meet mall santa came the world readi new carli rae jepsen album it four long year sinc although gotten cut to the feel plenti collab meantim but still world readi music close carli rae jepsen album thank exactli world get earlier month jepsen announc new album call dedic have song hit seen bitch play flute song hit the shoot it reason question it also relev question accompani footag woman pose first bust song flute solo break biggest viral danc craze backup dancer what gloriou sight behold here sing have ever seen bitch rosalía still play club heroin sing feel readi arena for first american club show worldwid el mal querer tour first ever north play headlin gig hit stage mayan lo angel show held wednesday two appear coachella place size muic may in the number one review everi singl singl histori billboard orchestra start chart begin work way present grand funk we an guitar piano hit septemb stay at week they realli american band they may american band grand funk railroad piano burst flint michigan la disput rut after play togeth give take member close year experimentalist stuck music familiar come idea upcom album in search someth anyth songwrit process la disput rut after play togeth give take music close year experimentalist stuck routin familiar come idea upcom album in search someth anyth songwrit process admittedli name intellexu bit hear the record good like realli good you expect much nico segal nate fox guy play key role craft chanc the rapper guitar sound count frank ocean kany west ultralight beam admittedli name intellexu bit hear"
#--- split the line into words ---
words = line.strip().split()
i=0
data=[]
# f=open('mapperTrial.txt','w')
    #--- output tuples [word, 1] in tab-delimited format---
for i in range(len(words)-1):
    if words[i] in topics:
    #     print('%s\t%s' % (word, "1"))
        w = words[i]+"|"+words[i+1]+"\t1\n"
        data.append(w)
    #     f.write(w)
    #     print(w.strip())
# f.close()
    
    
print(data)

['sing|one\t1\n', 'music|close\t1\n', 'song|hit\t1\n', 'song|hit\t1\n', 'song|flute\t1\n', 'sing|have\t1\n', 'sing|feel\t1\n', 'orchestra|start\t1\n', 'guitar|piano\t1\n', 'piano|hit\t1\n', 'piano|burst\t1\n', 'music|familiar\t1\n', 'music|close\t1\n', 'guitar|sound\t1\n']


In [52]:
import sys
 
# maps words to their counts
word2count = {}
 
# input comes from STDIN
for line in data:
    # remove leading and trailing whitespace
    line = line.strip()
 
    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)
    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        continue

    try:
        word2count[word] += count
    except:
        word2count[word] = count
 
 # write the tuples to stdout
 # Note: they are unsorted
# for word in word2count.keys():
#     print('%s\t%s'% ( word, word2count[word] ))
    
word2count= sorted(word2count.items(), key = 
             lambda kv:(kv[1], kv[0]),reverse=True)

for word in word2count:
    print('%s\t%s'% ( word[0], word[1] ))


song|hit	2
music|close	2
song|flute	1
sing|one	1
sing|have	1
sing|feel	1
piano|hit	1
piano|burst	1
orchestra|start	1
music|familiar	1
guitar|sound	1
guitar|piano	1
